In [1]:
using Revise, LatticeModels, BenchmarkTools, SparseArrays, StaticArrays

In [61]:
mutable struct SparseMatrixBuilder{T} <: LatticeModels.AbstractMatrixBuilder{T}
    sz::Tuple{Int,Int}
    maxcolsize::Int
    colsizes::Vector{Int}
    rowvals::Vector{Int}
    nzvals::Vector{T}
    function SparseMatrixBuilder{T}(szx::Int, szy::Int, col_hint::Int=4) where T
        col_hint = max(col_hint, 1)
        new{T}((szx, szy), col_hint, zeros(Int, szy), zeros(Int, col_hint * szy), zeros(T, col_hint * szy))
    end
end

function _grow_to!(b::SparseMatrixBuilder, new_maxcollen)
    new_rowvals = zeros(Int, (new_maxcollen, b.sz[2]))
    new_nzvals = similar(b.nzvals, (new_maxcollen, b.sz[2]))
    copyto!(@view(new_rowvals[1:b.maxcolsize, :]), b.rowvals)
    copyto!(@view(new_nzvals[1:b.maxcolsize, :]), b.nzvals)

    b.rowvals = vec(new_rowvals)
    b.nzvals = vec(new_nzvals)
    b.maxcolsize = new_maxcollen
    return b
end
Base.sizehint!(b::SparseMatrixBuilder, n::Int) = n > b.maxcolsize ? _grow_to!(b, n) : b

Base.@propagate_inbounds function Base.setindex!(b::SparseMatrixBuilder, x::Number, i::Int, j::Int; overwrite=true, factor=1)
    if b.colsizes[j] == b.maxcolsize
        _grow_to!(b, b.maxcolsize * 2)
    end
    col_start = (j - 1) * b.maxcolsize + 1
    col_end = col_start + b.colsizes[j] - 1
    I = col_start
    @inbounds for _ in 1:b.colsizes[j]
        b.rowvals[I] >= i && break
        I += 1
    end
    new_entry = b.rowvals[I] != i
    if b.rowvals[I] != i
        b.colsizes[j] += 1
        @inbounds for i in col_end:-1:I
            b.rowvals[i + 1] = b.rowvals[i]
            b.nzvals[i + 1] = b.nzvals[i]
        end
        b.rowvals[I] = i
    end
    if overwrite || new_entry
        b.nzvals[I] = x * factor
    else
        b.nzvals[I] += x * factor
    end
end

function to_matrix(b::SparseMatrixBuilder)
    mask = b.rowvals .== 0
    rowval = deleteat!(b.rowvals, mask)
    nzval = deleteat!(b.nzvals, mask)
    b.colsizes[1] += 1
    colptr = cumsum(b.colsizes)
    pushfirst!(colptr, 1)
    return SparseMatrixCSC(b.sz[1], b.sz[2], colptr, rowval, nzval)
end

to_matrix (generic function with 1 method)

In [142]:
l = SquareLattice(2000, 2000)
ll = addlookuptable(l)
ll2 = addlookuptable(setnnbonds(GenericLattice(ll), LatticeModels.getnnbonds(ll)))
nothing

In [149]:
@benchmark tightbinding_hamiltonian(ll, t1=3, t2=2, t3=1)

BenchmarkTools.Trial: 7 samples with 1 evaluation.
 Range (min … max):  765.953 ms … 901.923 ms  ┊ GC (min … max): 0.08% … 13.14%
 Time  (median):     782.834 ms               ┊ GC (median):    2.21%
 Time  (mean ± σ):   798.761 ms ±  47.317 ms  ┊ GC (mean ± σ):  3.35% ±  4.52%

  █ ██   █   █     █                                          █  
  █▁██▁▁▁█▁▁▁█▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  766 ms           Histogram: frequency by time          902 ms <

 Memory estimate: 1.19 GiB, allocs estimate: 77.

In [152]:
l3 = SquareLattice(2000, 200)

400000-site 2-dim Bravais lattice in 2D space
Unit cell:
  Basis site coordinates:
    ┌      ┐ 
    │ 0.000│ 
    │ 0.000│ 
    └      ┘ 
  Translation vectors:
    ┌      ┐ ┌      ┐ 
    │ 1.000│ │ 0.000│ 
    │ 0.000│ │ 1.000│ 
    └      ┘ └      ┘ 
Lattice type: SquareLattice{2}
Default translations: 
  :axis1 → Bravais[2000, 0]
  :axis2 → Bravais[0, 200]
Nearest neighbor hoppings: 
  1.00000 =>
    Bravais[1, 0]
    Bravais[0, 1]
  1.41421 =>
    Bravais[1, -1]
    Bravais[1, 1]
  2.00000 =>
    Bravais[2, 0]
    Bravais[0, 2]
Boundary conditions: none

In [147]:
@benchmark tightbinding_hamiltonian(ll2, t1=3, t2=2, t3=1)

BenchmarkTools.Trial: 6 samples with 1 evaluation.
 Range (min … max):  778.529 ms …    1.168 s  ┊ GC (min … max): 0.08% … 10.36%
 Time  (median):     949.568 ms               ┊ GC (median):    2.52%
 Time  (mean ± σ):   976.148 ms ± 154.397 ms  ┊ GC (mean ± σ):  3.44% ±  3.75%

  █              ██                   █                  █    █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁█ ▁
  779 ms           Histogram: frequency by time          1.17 s <

 Memory estimate: 1.19 GiB, allocs estimate: 44.

In [148]:
using Profile, PProf
Profile.clear()
@pprof for _ in 1:10; tightbinding_hamiltonian(UniformMatrixBuilder{ComplexF64}, ll2, t1=3, t2=2, t3=1); end

"profile.pb.gz"

In [ ]:
using Profile, PProf
Profile.Allocations.clear()
@pprof for _ in 1:10; tightbinding_hamiltonian(UniformMatrixBuilder{ComplexF64}, ll, t1=3, t2=2, t3=1); end

In [106]:
using SparseArrays
function hop_ff!(b, l, hop::LatticeModels.AbstractTranslation, factor=1)
    i = 0
    for site in l
        i += 1
        rs2 = LatticeModels.resolve_site(l, site + hop)
        if rs2 === nothing
            continue
        end
        b[i, rs2.index] = factor
        b[rs2.index, i] = factor
    end
end
function tb_ff(l; t1 = 1, t2 = 0, t3 = 0)
    nns = Tuple(LatticeModels.adapt_bonds(NearestNeighbor(i), l) for i in 1:3)
    ts = (t1, t2, t3)
    le = sum(length, (nn.translations for nn in nns)) * 2
    b = SparseMatrixBuilder{ComplexF64}(length(l), length(l), le)
    for i in 1:3
        for hop in nns[i].translations
            hop_ff!(b, l, hop, ts[i])
        end
    end
    mat = to_matrix(b)
    return Hamiltonian(System(l), mat)
end

tb_ff (generic function with 1 method)

In [111]:
mat = tb_ff(ll, t1=3, t2=2, t3=1).data
@benchmark Hamiltonian(System(ll), mat)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.400 μs …   7.496 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.533 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.592 μs ± 239.465 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁▅  ▃█▂ ▄                                                    
  ██▃▂███▇█▇▅▄▅▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  1.4 μs          Histogram: frequency by time        2.52 μs <

 Memory estimate: 2.00 KiB, allocs estimate: 18.

In [107]:
@benchmark tb_ff(ll, t1=3, t2=2, t3=1)

BenchmarkTools.Trial: 7 samples with 1 evaluation.
 Range (min … max):  661.948 ms … 822.404 ms  ┊ GC (min … max): 0.11% … 14.68%
 Time  (median):     713.865 ms               ┊ GC (median):    1.91%
 Time  (mean ± σ):   723.639 ms ±  49.540 ms  ┊ GC (mean ± σ):  3.60% ±  5.09%

  ▁           ▁      █   ▁   ▁                                ▁  
  █▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁█▁▁▁█▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  662 ms           Histogram: frequency by time          822 ms <

 Memory estimate: 1.19 GiB, allocs estimate: 86.

In [101]:
@pprof for _ in 1:10; tb_ff(ll, t1=3, t2=2, t3=1); end

"profile.pb.gz"

In [109]:
tb_ff(ll, t1=3, t2=2, t3=1) ≈ tightbinding_hamiltonian(ll, t1=3, t2=2, t3=1)

true

## DictMatrixBuilder

In [12]:
using Dictionaries
struct DictMatrixBuilder{T} <: LatticeModels.AbstractMatrixBuilder{T}
    sz::Tuple{Int,Int}
    dict::Dictionary{Int, T}
    function DictMatrixBuilder{T}(szx::Int, szy::Int, colhint::Int=1) where T
        d = Dictionary{Int, T}()
        # sizehint!(d, colhint * szy)
        new{T}((szx, szy), d)
    end
end

Base.@propagate_inbounds function Base.setindex!(b::DictMatrixBuilder, x, i::Int, j::Int; overwrite=true, factor=1)
    ind = i + (j - 1) * b.sz[1]
    hadkey, token = gettoken!(b.dict, ind)
    if hadkey && !overwrite
        val = gettokenvalue(b.dict, token)
        settokenvalue!(b.dict, token, val + x * factor)
    else
        settokenvalue!(b.dict, token, x * factor)
    end
end

function LatticeModels.to_matrix(b::DictMatrixBuilder{T}) where T
    len = length(b.dict)
    nzval = collect(values(b.dict))
    indval = collect(keys(b.dict))
    rowval = Array{Int}(undef, len)
    colval = Array{Int}(undef, len)
    @simd for I in 1:len
        j, i = divrem(indval[I] - 1, b.sz[1]) .+ 1
        rowval[I] = i
        colval[I] = j
    end
    sparse(rowval, colval, nzval, b.sz...)
end

In [34]:
ls = SquareLattice(100, 100)
lls = addlookuptable(ls)
H = tightbinding_hamiltonian(lls, t1=3, t2=2, t3=1)
H2 = tightbinding_hamiltonian(SimpleMatrixBuilder{ComplexF64}, lls, t1=3, t2=2, t3=1)
H == H2

true

In [4]:
using SparseMatrixDicts
struct SMDMatrixBuilder{T} <: LatticeModels.AbstractMatrixBuilder{T}
    builder::SparseMatrixDict{T, Int}
    function SMDMatrixBuilder{T}(szx::Int, szy::Int, colhint::Int=1) where T
        new{T}(SparseMatrixDict{T, Int}(szx, szy))
    end
end

Base.@propagate_inbounds function Base.setindex!(b::SMDMatrixBuilder, x::Number, i::Int, j::Int; overwrite=true, factor=1)
    if overwrite
        b.builder[i, j] = x * factor
    else
        b.builder[i, j] += x * factor
    end
end

function LatticeModels.to_matrix(b::SMDMatrixBuilder{T}) where T
    sparse(b.builder)
end

## Comparing matrix builders

In [1]:
using BenchmarkTools, Revise, LatticeModels

In [83]:
l = SquareLattice(2000, 2000)
ll = addlookuptable(l)

4000000-site 2-dim Bravais lattice in 2D space
Unit cell:
  Basis site coordinates:
    ┌      ┐ 
    │ 0.000│ 
    │ 0.000│ 
    └      ┘ 
  Translation vectors:
    ┌      ┐ ┌      ┐ 
    │ 1.000│ │ 0.000│ 
    │ 0.000│ │ 1.000│ 
    └      ┘ └      ┘ 
Lattice type: SquareLattice{2}
Default translations: 
  :axis1 → Bravais[2000, 0]
  :axis2 → Bravais[0, 2000]
Nearest neighbor hoppings: 
  1.00000 =>
    Bravais[1, 0]
    Bravais[0, 1]
  1.41421 =>
    Bravais[1, -1]
    Bravais[1, 1]
  2.00000 =>
    Bravais[2, 0]
    Bravais[0, 2]
Boundary conditions: none
Lookup table: 4000000 sites, 2000 strides, secondary keys enabled

In [84]:
@benchmark tightbinding_hamiltonian(ll, t1=3, t2=2, t3=1)

BenchmarkTools.Trial: 6 samples with 1 evaluation.
 Range (min … max):  836.059 ms …   1.086 s  ┊ GC (min … max): 2.07% … 0.09%
 Time  (median):     880.557 ms              ┊ GC (median):    1.85%
 Time  (mean ± σ):   915.835 ms ± 99.489 ms  ┊ GC (mean ± σ):  4.05% ± 5.79%

  ▁█                 ▁            ▁                          ▁  
  ██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  836 ms          Histogram: frequency by time          1.09 s <

 Memory estimate: 1.19 GiB, allocs estimate: 91.

In [85]:
@benchmark tightbinding_hamiltonian(UniformMatrixBuilder{ComplexF64}, ll, t1=3, t2=2, t3=1, col_hint=12)

BenchmarkTools.Trial: 6 samples with 1 evaluation.
 Range (min … max):  794.618 ms … 917.023 ms  ┊ GC (min … max): 1.58% … 0.17%
 Time  (median):     821.322 ms               ┊ GC (median):    1.51%
 Time  (mean ± σ):   842.132 ms ±  55.446 ms  ┊ GC (mean ± σ):  3.33% ± 5.06%

  █   ▁                 ▁                               ▁     ▁  
  █▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁█ ▁
  795 ms           Histogram: frequency by time          917 ms <

 Memory estimate: 1.19 GiB, allocs estimate: 132.

In [86]:
@benchmark tightbinding_hamiltonian(SimpleMatrixBuilder{ComplexF64}, ll, t1=3, t2=2, t3=1)

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  922.274 ms …    2.027 s  ┊ GC (min … max):  1.35% … 17.15%
 Time  (median):        1.411 s               ┊ GC (median):    16.06%
 Time  (mean ± σ):      1.443 s ± 480.735 ms  ┊ GC (mean ± σ):  14.10% ±  7.67%

  █               █                    █                      █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  922 ms           Histogram: frequency by time          2.03 s <

 Memory estimate: 3.66 GiB, allocs estimate: 144.

In [87]:
@benchmark tightbinding_hamiltonian(SimpleMatrixBuilder{LatticeModels.ArrayEntry{ComplexF64}}, ll, t1=3, t2=2, t3=1)

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  1.797 s …    2.406 s  ┊ GC (min … max):  0.70% … 16.13%
 Time  (median):     2.187 s               ┊ GC (median):    14.30%
 Time  (mean ± σ):   2.130 s ± 308.399 ms  ┊ GC (mean ± σ):  11.16% ±  8.43%

  █                                    █                   █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.8 s          Histogram: frequency by time         2.41 s <

 Memory estimate: 5.45 GiB, allocs estimate: 147.

In [88]:
@benchmark tightbinding_hamiltonian(DictMatrixBuilder{ComplexF64}, ll, t1=3, t2=2, t3=1)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 10.451 s (5.65% GC) to evaluate,
 with a memory estimate of 11.25 GiB, over 300 allocations.

In [24]:
@benchmark tightbinding_hamiltonian(SMDMatrixBuilder{ComplexF64}, ll, t1=3, t2=2, t3=1)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 10.505 s (6.84% GC) to evaluate,
 with a memory estimate of 12.62 GiB, over 242 allocations.